In [ ]:
from quchem.Hamiltonian_Generator_Functions import *
from quchem.Graph import *
### HAMILTONIAN start
Molecule = 'LiH'
geometry = None # [('H', (0., 0., 0.)), ('H', (0., 0., 0.74))]
basis = 'sto-6g'


### Get Hamiltonian
Hamilt = Hamiltonian_PySCF(Molecule,
                     run_scf=1, run_mp2=1, run_cisd=1, run_ccsd=1, run_fci=1,
                     basis=basis,
                     multiplicity=1,
                     geometry=geometry)  # normally None!
QubitHamiltonian = Hamilt.Get_Qubit_Hamiltonian(threshold=None, transformation='BK')
### HAMILTONIAN end
QubitHamiltonian

The LiH ground state under JW is:

$$|\psi \rangle_{HF}^{ground} = | 1 1 1 1 \:  0 0 0 0 0 0 0 0 \rangle$$

and under BK is:

$$|\psi \rangle_{HF}^{ground} = | 1 0 1 0 0 0 0 0 0 0 0 0 \rangle$$

Use NOON to reduces this:

In [ ]:
from quchem.Ansatz_Generator_Functions import *

##
NOON_spins_combined, NMO_basis = Hamilt.Get_NOON()
##
Hamilt.Get_CCSD_Amplitudes()

NOON_spins_combined

LOOK at PHYS. REV. X, **8**, 031022 (2018)

Due to NOON:
- indices 0,1 assumed to ALWAYS be filled
- indices 8,9 and 10,11 ALWAYS unfilled! <--- decision



# Therefore Simplify Hamiltonian

In [ ]:
ansatz_obj = BK_Qubit_Reduction(QubitHamiltonian,
                             Hamilt.molecule.n_electrons,
                             Hamilt.molecule.n_qubits)

The LiH ground state under JW is:

$$|\psi \rangle_{HF}^{ground} = | 1 1 1 1 \:  0 0 0 0 0 0 0 0 \rangle$$

and under BK is:

$$|\psi \rangle_{HF}^{ground} = | 1 0 1 0 0 0 0 0 0 0 0 0 \rangle$$


we have **fixed occupations** of indices [0,1,8,9,10,11]

- therefore can remove these terms from the Hamiltonian!

In [ ]:
list_of_qubit_indices_to_remove = [0,1,8,9,10,11]
reduced_Qubit_Hamiltonian = ansatz_obj.Remove_indices_from_Hamiltonian(list_of_qubit_indices_to_remove)
print('Hamiltonian size reduced: {} --> {}'.format(len(list(QubitHamiltonian)), len(list(reduced_Qubit_Hamiltonian))))

In [ ]:
qubit_re_label_dict, reduced_RE_LABELLED_Qubit_Hamiltonian = ansatz_obj.Re_label_Hamiltonian(reduced_Qubit_Hamiltonian)
reduced_RE_LABELLED_Qubit_Hamiltonian

In [6]:
new_Molecular_H_MATRIX = Hamilt.Get_sparse_Qubit_Hamiltonian_matrix(reduced_RE_LABELLED_Qubit_Hamiltonian)


from scipy.sparse.linalg import eigs
eig_values, eig_vectors = eigs(new_Molecular_H_MATRIX)
new_FCI_Energy = min(eig_values)


print('new_FCI = ', new_FCI_Energy, 'VS old FCI:', Hamilt.molecule.fci_energy)

new_FCI =  (-7.940677175840136+1.1005065191298234e-16j) VS old FCI: -7.875652564927912


## Looking at Ansatz

In [7]:
# automate:
# reduced_Sec_Quant_CC_ops_ia, reduced_Sec_Quant_CC_ops_ijab, reduced_theta_parameters_ia, reduced_theta_parameters_ijab =ansatz_obj.Remove_NOON_terms(
#     NOON=NOON_spins_combined,
#     occ_threshold= 1.999,
#     unocc_threshold=1e-4,
#     indices_to_remove_list_manual=None, 
#     single_cc_amplitudes=Hamilt.molecule.single_cc_amplitudes,
#     double_cc_amplitudes=Hamilt.molecule.double_cc_amplitudes,
#     singles_hamiltonian=Hamilt.singles_hamiltonian,
#     doubles_hamiltonian=Hamilt.doubles_hamiltonian,
#     tol_filter_small_terms=None)

# manual
reduced_Sec_Quant_CC_ops_ia, reduced_Sec_Quant_CC_ops_ijab, reduced_theta_parameters_ia, reduced_theta_parameters_ijab =ansatz_obj.Remove_NOON_terms(
    NOON=NOON_spins_combined,
    indices_to_remove_list_manual=[0,1,8,9,10,11])

In [8]:
ia_terms, ijab_terms, ia_theta, ijab_theta = ansatz_obj.Get_ia_and_ijab_terms()
print('REDUCTION')
print('ia_terms', len(ia_terms), 'TO', len(reduced_Sec_Quant_CC_ops_ia))
print('ijab_terms', len(ijab_terms), 'TO', len(reduced_Sec_Quant_CC_ops_ijab))

REDUCTION
ia_terms 16 TO 4
ijab_terms 42 TO 3


In [9]:
Qubit_Op_list_Second_Quant_CC_Ops_ia, Qubit_Op_list_Second_Quant_CC_Ops_ijab = ansatz_obj.UCCSD_single_trotter_step(reduced_Sec_Quant_CC_ops_ia,
                                                                                                                    reduced_Sec_Quant_CC_ops_ijab,
                                                                                                                    transformation='BK')

# # NO reduction!
# Qubit_Op_list_Second_Quant_CC_Ops_ia, Qubit_Op_list_Second_Quant_CC_Ops_ijab = ansatz_obj.UCCSD_single_trotter_step(ia_terms,
#                                                                                                                     ijab_terms,
#                                                                                                                     transformation='BK')


In [10]:
reduced_CC_ijab = ansatz_obj.Remove_indices_from_CC_qubit_operators(Qubit_Op_list_Second_Quant_CC_Ops_ijab, list_of_qubit_indices_to_remove)
len(reduced_CC_ijab)

3

In [11]:
reduced_RE_LABELLED_CC_ijab = ansatz_obj.Re_label_CC_qubit_operators( qubit_re_label_dict, 
                                                                   reduced_CC_ijab)
reduced_RE_LABELLED_CC_ijab

[0.125j [X0 Z1 Y2] +
 0.125j [X0 Z1 Y2 Z3] +
 0.125j [X0 Y2] +
 0.125j [X0 Y2 Z3] +
 -0.125j [Y0 Z1 X2] +
 -0.125j [Y0 Z1 X2 Z3] +
 -0.125j [Y0 X2] +
 -0.125j [Y0 X2 Z3],
 0.125j [X0 Z1 X2 Y3 Z4] +
 0.125j [X0 Z1 Y2 X3 Z5] +
 0.125j [X0 X2 Y3 Z4] +
 0.125j [X0 Y2 X3 Z5] +
 -0.125j [Y0 Z1 X2 X3 Z5] +
 0.125j [Y0 Z1 Y2 Y3 Z4] +
 -0.125j [Y0 X2 X3 Z5] +
 0.125j [Y0 Y2 Y3 Z4],
 0.125j [X0 Z1 Z3 Y4 Z5] +
 0.125j [X0 Z1 Y4] +
 0.125j [X0 Z3 Y4 Z5] +
 0.125j [X0 Y4] +
 -0.125j [Y0 Z1 Z3 X4 Z5] +
 -0.125j [Y0 Z1 X4] +
 -0.125j [Y0 Z3 X4 Z5] +
 -0.125j [Y0 X4]]

In [12]:
reduced_CC_ia = ansatz_obj.Remove_indices_from_CC_qubit_operators(Qubit_Op_list_Second_Quant_CC_Ops_ia, list_of_qubit_indices_to_remove)

reduced_RE_LABELLED_CC_ia = ansatz_obj.Re_label_CC_qubit_operators( qubit_re_label_dict, 
                                                                   reduced_CC_ia)
reduced_RE_LABELLED_CC_ia

[0.5j [X0 Y1 X2] +
 0.5j [Y0 Y1 Y2],
 0.5j [X0 Y1 Z3 X4] +
 0.5j [Y0 Y1 Z3 Y4],
 0.5j [Z0 Y1 Z2 X3] +
 -0.5j [X1 Y3],
 0.5j [Z0 Y1 Z3 Z4] +
 -0.5j [Y1 Z5]]

## HF input state

In [13]:
print('old input = ', ansatz_obj.Get_BK_HF_state_in_OCC_basis())

print('BUT following indices removed:', list_of_qubit_indices_to_remove)

ansatz_obj.New_BK_HF_state(list_of_qubit_indices_to_remove)

old input =  [1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
BUT following indices removed: [0, 1, 8, 9, 10, 11]


array([1., 0., 0., 0., 0., 0.])

# Ansatz

look at: From PHYS. REV. X, **8**, 031022 (2018)

can make a further reduction!

In [14]:
import random
theta_ia_random_input = [random.uniform(0, 2*np.pi) for _ in range(len(reduced_RE_LABELLED_CC_ia))]
theta_ijab_random_input = [random.uniform(0, 2*np.pi) for _ in range(len(reduced_RE_LABELLED_CC_ijab))]

In [15]:
input_state = ansatz_obj.New_BK_HF_state(list_of_qubit_indices_to_remove)
print('input_state =', input_state)

UCCSD_ansatz_Q_Circ_obj = Ansatz_Circuit(input_state,
                                     reduced_RE_LABELLED_CC_ia, 
                                     reduced_RE_LABELLED_CC_ijab)

ansatz_cirq_circuit =UCCSD_ansatz_Q_Circ_obj.Get_Full_HF_UCCSD_QC(Theta_param_list_ia=theta_ia_random_input, 
                                         Theta_param_list_ijab=theta_ijab_random_input,
                                         ia_first=True)
ansatz_cirq_circuit

input_state = [1. 0. 0. 0. 0. 0.]


┌─────────┐   ┌────────────┐                ┌──────────┐
0: ───X───H──────────@────────────────────────@───H───────────Rx(0.5π)───@────────────────────────@───────────Rx(-0.5π)───H──────────@─────────────────────────────────@──────────H───────────Rx(0.5π)───@─────────────────────────────────────────@───Rx(-0.5π)──────────────@─────────────────────────────────@──────────────────────────────────────────────────────────────────────@─────────────────────────────────@───H────────────────────@─────────────────────@─────────────H─────────────H───────────@─────────────────────────@───────────H───Rx(0.5π)───@────────────────@───Rx(-0.5π)───Rx(0.5π)───@────────────────────────@───Rx(-0.5π)───Rx(0.5π)───@────────────────────────@───Rx(-0.5π)───Rx(0.5π)───@────────────────────────────────@───Rx(-0.5π)───H───@─────────────────────────@───H───────────H──────────@────────────────────────────────────────@───────────H───H───@─────────────────────────────────@───────────H──────────H───@─────────────────────────────────────────────────@──────────H──────────Rx(0.5π)───@─────────────────────────────────@───────────Rx(-0.5π)───Rx(0.5π)───@────────────────────────────────────────────────────────@───────────Rx(-0.5π)───Rx(0.5π)───@────────────────────────────────────────@───Rx(-0.5π)───Rx(0.5π)───@────────────────────────────────@───Rx(-0.5π)───H──────────@─────────────────────────────────────────@───────────H──────────H───@─────────────────────────────────@───H───────────H───@─────────────────@───H───────────H──────────@─────────────────────────@───────────H───Rx(0.5π)───@────────────────@───Rx(-0.5π)───Rx(0.5π)───@────────────────────────@───Rx(-0.5π)───Rx(0.5π)───@────────────────────────────────────────@───Rx(-0.5π)───Rx(0.5π)───@────────────────────────────────@───Rx(-0.5π)───H───@─────────────────────────────────────────────────@──────────H───H───@─────────────────────────────────@───────────H───
                     │                        │                          │                        │                                  │                                 │                                 │                                         │                          │                                 │                                                                      │                                 │                        │                     │                                       │                         │                          │                │                          │                        │                          │                        │                          │                                │                   │                         │                          │                                        │                   │                                 │                          │                                                 │                                │                                 │                                  │                                                        │                                  │                                        │                          │                                │                          │                                         │                          │                                 │                   │                 │                          │                         │                          │                │                          │                        │                          │                                        │                          │                                │                   │                                                 │                  │                                 │
1: ───I───Rx(0.5π)───X───@────────────────@───X───Rx(-0.5π)───Rx(0.5π)───X───@────────────────@───X───────────Rx(-0.5π)───Rx(0.5π)───X───@─────────────────────────@───X──────────Rx(-0.5π)───Rx(0.5π)───X───@─────────────────────────@───────────X───R

# Running Example

In [16]:
Hamiltonian_graph_obj = Openfermion_Hamiltonian_Graph(reduced_RE_LABELLED_Qubit_Hamiltonian)

commutativity_flag = 'AC' ## <- defines relationship between sets!!!
plot_graph = False
Graph_colouring_strategy='largest_first'
anti_commuting_sets = Hamiltonian_graph_obj.Get_Clique_Cover_as_QubitOp(commutativity_flag, Graph_colouring_strategy=Graph_colouring_strategy, plot_graph=plot_graph)
anti_commuting_sets

Building Graph Edges: 100%|##########| 239/239 [00:02<00:00, 97.43it/s] 


{0: [(0.0782811414931556+0j) [Z1 Z3 Z5]],
 1: [(-7.26483183797169+0j) []],
 2: [(-0.14432019122698908+0j) [Z1 Z3 Z4 Z5],
  (0.005610346244611411+0j) [Y4],
  (0.010671449192243557+0j) [Z1 Z3 X4 Z5]],
 3: [(-0.14432019122698903+0j) [Z4],
  (0.005610346244611411+0j) [Z1 Z3 Y4 Z5],
  (0.010671449192243557+0j) [X4]],
 4: [(0.017548783925032813+0j) [Z0],
  (0.004931982511594008+0j) [Y0],
  (0.10945352587995585+0j) [X0]],
 5: [(-0.13431719877572376+0j) [Z2 Z3],
  (0.004739070747220776+0j) [X1 Z2 X3],
  (0.011871946723161789+0j) [Y2],
  (0.022273980559951535+0j) [X2],
  (0.0025594391105535136+0j) [X1 Z2 Y3 X4 Z5],
  (0.03287120818002574+0j) [Z1 Z2 X3 Y5],
  (0.0025594391105535136+0j) [X1 Z2 Y3 Y4 Z5],
  (-0.001044195770483135+0j) [Y1 Z2 X3 Z4]],
 6: [(-0.13431719877572376+0j) [Z2],
  (0.011871946723161789+0j) [Y2 Z3],
  (0.0058630615794339844+0j) [X1 Y2 X3],
  (0.004739070747220773+0j) [X0 Y1 Y2 X3],
  (-0.002688933646884141+0j) [Z1 Y2 X3 Y5],
  (0.004836443728007132+0j) [Z0 X1 Y2 Y3 X4 Z5],
 

In [17]:
from quchem.LCU_method import *

In [18]:
N_QUBITS = Hamilt.molecule.n_qubits - len(list_of_qubit_indices_to_remove)

def GIVE_ENERGY_lin_alg(theta_ia_ijab):
    
    theta_ia= theta_ia_ijab[:(len(reduced_theta_parameters_ia))]
    theta_ijab=theta_ia_ijab[(len(reduced_theta_parameters_ia)):]
   
    ansatz_cirq_circuit =UCCSD_ansatz_Q_Circ_obj.Get_Full_HF_UCCSD_QC(
                                        Theta_param_list_ia=theta_ia, 
                                         Theta_param_list_ijab=theta_ijab,
                                         ia_first=True)

    VQE_exp_LCU_lin_alg = VQE_Experiment_LCU_UP_lin_alg(anti_commuting_sets,
                 ansatz_cirq_circuit,
                 N_QUBITS,                     # <--- NOTE THIS
                 N_indices_dict=None) 
    
    return VQE_exp_LCU_lin_alg.Calc_Energy().real

In [19]:
import random

theta_ia_random_input = [random.uniform(0, 2*np.pi) for _ in range(len(reduced_theta_parameters_ia))]
theta_ijab_random_input = [random.uniform(0, 2*np.pi) for _ in range(len(reduced_theta_parameters_ijab))]

theta_combined_random_input = [*theta_ia_random_input, *theta_ijab_random_input]

GIVE_ENERGY_lin_alg(theta_combined_random_input)

-7.202325526518836

## Simulating

In [ ]:
theta_combined_random_input

In [ ]:
def calc_gradient_ADAM(theta_ia_theta_jab_list):
    
    grad_list=[]
    
    for index, theta in enumerate(theta_ia_theta_jab_list):
        
        new_theta_list = theta_ia_theta_jab_list.copy()
        new_theta_list[index] = theta + np.pi/4
        
        Obs_PLUS = GIVE_ENERGY_lin_alg(new_theta_list)
               
        
        new_theta_list[index] = theta - np.pi/4
        Obs_MINUS = GIVE_ENERGY_lin_alg(new_theta_list)
        
        gradient = Obs_PLUS - Obs_MINUS
        
        grad_list.append(gradient)
        

    return np.array(grad_list)

In [ ]:
### optimizer
from quchem.Scipy_Optimizer import _minimize_Adam, Optimizer

In [ ]:
custom_optimizer_DICT = {'learning_rate': 0.1, 'beta_1': 0.9, 'beta_2': 0.999, 'epsilon': 1e-8,
                           'delta': 1e-8, 'maxfev': 15000}

GG = Optimizer(GIVE_ENERGY_lin_alg,
                   theta_combined_random_input,
                   args=(),
                   method=_minimize_Adam,
                   jac=calc_gradient_ADAM,
                   hess=None,
                   hessp=None,
                      bounds=None,
                   constraints=None,
                   tol=1e-20, 
                   display_convergence_message=True,
                   display_steps=True, 
                   custom_optimizer_DICT=custom_optimizer_DICT)
GG.get_env(100)
GG.plot_convergence()
plt.show()

In [ ]:
# Nelder-Mead

GG = Optimizer(GIVE_ENERGY_lin_alg,
                   theta_combined_random_input,
                   args=(),
                   method='Nelder-Mead',
                   jac=None,
                   hess=None,
                   hessp=None,
                      bounds=None,
                   constraints=None,
                   display_convergence_message=True,
                   display_steps=True)

GG.get_env(50)
GG.plot_convergence()
plt.show()

In [ ]:
# # finite difference

# custom_optimizer_DICT = {'learning_rate': 0.1, 'beta_1': 0.9, 'beta_2': 0.999, 'epsilon': 1e-8,
#                            'delta': 1e-8, 'maxfev': 15000}

# GG = Optimizer(GIVE_ENERGY_lin_alg,
#                    theta_combined_random_input,
#                    args=(),
#                    method=_minimize_Adam,
#                    jac=None,                 # <-- finite difference approach!
#                    hess=None,
#                    hessp=None,
#                       bounds=None,
#                    constraints=None,
#                    tol=1e-20, 
#                    display_convergence_message=True,
#                    display_steps=True, 
#                    custom_optimizer_DICT=custom_optimizer_DICT)
# GG.get_env(100)
# GG.plot_convergence()
# plt.show()

In [ ]:
Hamilt.molecule.fci_energy

In [ ]:
HF_state_prep = State_Prep(input_STATE)
HF_state_prep_circuit = cirq.Circuit(cirq.decompose_once(
    (HF_state_prep(*cirq.LineQubit.range(HF_state_prep.num_qubits())))))
print(HF_state_prep_circuit)

In [ ]:
input_STATE = ansatz_obj.New_BK_HF_state(list_of_qubit_indices_to_remove)

xx = Ansatz_Circuit_new(input_STATE, reduced_RE_LABELLED_CC_ia, reduced_RE_LABELLED_CC_ijab)

xx.Get_Full_HF_UCCSD_QC(Theta_param_list_ia=None, Theta_param_list_ijab=[5,6,7])